In [1]:
import pandas as pd

In [2]:
issues = pd.read_pickle('../../data/01_data_from_db/issues.pkl')
issues.head()

,issue_id,project_id,created_at
0,1,4,2012-10-10 07:53:00
1,2,4,2012-06-20 15:23:42
2,3,3,2013-05-15 12:48:49
3,4,3,2013-01-29 17:08:05
4,5,3,2013-01-21 17:37:35


In [3]:
issue_comments = pd.read_pickle('../../data/01_data_from_db/issue_comments.pkl')
issue_comments.head()

,issue_id,comment_id,created_at
0,2,6460008,2012-06-20 16:41:07
1,2,6462188,2012-06-20 17:59:48
2,2,6464892,2012-06-20 19:40:23
3,2,6466645,2012-06-20 20:44:10
4,2,6562256,2012-06-25 22:43:17


In [4]:
issues_with_comments = pd.merge(issues, issue_comments, on=['issue_id'], sort=False) \
    .drop(columns={'created_at_x'}) \
    .rename(columns={'created_at_y': 'created_at'})
issues_with_comments

,issue_id,project_id,comment_id,created_at
0,2,4,6460008,2012-06-20 16:41:07
1,2,4,6462188,2012-06-20 17:59:48
2,2,4,6464892,2012-06-20 19:40:23
3,2,4,6466645,2012-06-20 20:44:10
4,2,4,6562256,2012-06-25 22:43:17
...,...,...,...,...
534054,154988,79163,25224474,2013-09-27 05:54:27
534055,154988,79163,25224484,2013-09-27 05:54:48
534056,154990,79163,25175557,2013-09-26 15:09:38
534057,154990,79163,25177616,2013-09-26 15:33:17


## Nowe issues w miesiącu

In [5]:
issues['year'] = issues['created_at'].dt.year
issues['month'] = issues['created_at'].dt.month
issues.head()

,issue_id,project_id,created_at,year,month
0,1,4,2012-10-10 07:53:00,2012,10
1,2,4,2012-06-20 15:23:42,2012,6
2,3,3,2013-05-15 12:48:49,2013,5
3,4,3,2013-01-29 17:08:05,2013,1
4,5,3,2013-01-21 17:37:35,2013,1


In [6]:
new_issues = issues.groupby(['project_id', 'year', 'month']).count().reset_index() \
    .rename(columns={'created_at': 'new_issues'}) \
    .drop(columns={'issue_id'})
new_issues.head()

,project_id,year,month,new_issues
0,1,2009,7,46
1,1,2009,8,9
2,1,2009,9,1
3,1,2009,10,3
4,1,2010,9,6


## Liczba wszystkich issues od początku projektu

In [7]:
total_issues_list = []
for index, row in new_issues.iterrows():
    total_issues_list.append(new_issues[(((row['year'] > new_issues['year']) |
        ((row['year'] == new_issues['year']) & (row['month'] >= new_issues['month']))) &
        (row['project_id'] == new_issues['project_id']))]['new_issues'].sum())

In [8]:
new_issues['total_issues'] = pd.Series(total_issues_list)

Sprawdzenie dla jednego z projektów

In [9]:
new_issues[new_issues['project_id'] == 3]

,project_id,year,month,new_issues,total_issues
71,3,2010,11,2,2
72,3,2011,2,2,4
73,3,2012,4,1,5
74,3,2012,6,1,6
75,3,2013,1,4,10
76,3,2013,5,2,12


In [10]:
new_issues.to_pickle('../../data/02_prepared_data/new_issues.pkl')

## Nowe komentarze issues w miesiącu

In [11]:
issues_with_comments['year'] = issues_with_comments.created_at.dt.year
issues_with_comments['month'] = issues_with_comments.created_at.dt.month
issues_with_comments.head()

,issue_id,project_id,comment_id,created_at,year,month
0,2,4,6460008,2012-06-20 16:41:07,2012,6
1,2,4,6462188,2012-06-20 17:59:48,2012,6
2,2,4,6464892,2012-06-20 19:40:23,2012,6
3,2,4,6466645,2012-06-20 20:44:10,2012,6
4,2,4,6562256,2012-06-25 22:43:17,2012,6


In [12]:
new_issue_comments = issues_with_comments.groupby(['project_id', 'year', 'month']).count().reset_index().drop(columns={'issue_id', 'comment_id'}).rename(columns={'created_at': 'new_issue_comments'})
new_issue_comments.head()

,project_id,year,month,new_issue_comments
0,1,2009,7,14
1,1,2009,8,12
2,1,2009,9,1
3,1,2009,10,5
4,1,2010,11,3


## Łączna liczba nowych komentarzy issues

In [13]:
total_issue_comments_list = []
for index, row in new_issue_comments.iterrows():
    total_issue_comments_list.append(new_issue_comments[(((row['year'] > new_issue_comments['year']) |
        ((row['year'] == new_issue_comments['year']) & (row['month'] >= new_issue_comments['month']))) &
        (row['project_id'] == new_issue_comments['project_id']))]['new_issue_comments'].sum())

In [14]:
new_issue_comments['total_issue_comments'] = pd.Series(total_issue_comments_list)

Sprawdzenie dla jednego z projektów

In [15]:
new_issue_comments[new_issue_comments['project_id'] == 3]

,project_id,year,month,new_issue_comments,total_issue_comments
68,3,2010,11,5,5
69,3,2010,12,3,8
70,3,2011,2,7,15
71,3,2011,3,2,17
72,3,2012,4,3,20
73,3,2012,6,5,25
74,3,2013,1,1,26
75,3,2013,2,3,29
76,3,2013,5,1,30


In [16]:
new_issue_comments.to_pickle('../../data/02_prepared_data/new_issue_comments.pkl')